In [1]:
# Week1_Project

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("customer_signups.csv")
df.head()

,customer_id,name,email,signup_date,source,region,plan_selected,marketing_opt_in,age,gender
0,CUST00000,Joshua Bryant,NaN,NaN,Instagram,NaN,basic,No,34,Female
1,CUST00001,Nicole Stewart,nicole1@example.com,02-01-24,LinkedIn,West,basic,Yes,29,Male
2,CUST00002,Rachel Allen,rachel2@example.com,03-01-24,Google,North,PREMIUM,Yes,34,Non-Binary
3,CUST00003,Zachary Sanchez,zachary3@mailhub.org,04-01-24,YouTube,NaN,Pro,No,40,Male
4,CUST00004,NaN,matthew4@mailhub.org,05-01-24,LinkedIn,West,Premium,No,25,Other


In [3]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       298 non-null    object
 1   name              291 non-null    object
 2   email             266 non-null    object
 3   signup_date       298 non-null    object
 4   source            291 non-null    object
 5   region            270 non-null    object
 6   plan_selected     292 non-null    object
 7   marketing_opt_in  290 non-null    object
 8   age               288 non-null    object
 9   gender            292 non-null    object
dtypes: object(10)
memory usage: 23.6+ KB


customer_id          2
name                 9
email               34
signup_date          2
source               9
region              30
plan_selected        8
marketing_opt_in    10
age                 12
gender               8
dtype: int64

In [4]:
import pandas as pd
import numpy as np

# Reload fresh (optional but recommended)
df = pd.read_csv("customer_signups.csv")

# 1) Drop rows with missing customer_id (can’t track user without it)
df = df.dropna(subset=["customer_id"])

# 2) Convert signup_date to datetime
df["signup_date"] = pd.to_datetime(df["signup_date"], errors="coerce", dayfirst=True)

# 3) Standardise text columns
df["source"] = df["source"].astype(str).str.strip().str.title().replace({"Nan": np.nan})
df["region"] = df["region"].astype(str).str.strip().str.title().replace({"Nan": np.nan})

# Standardise plan values
df["plan_selected"] = df["plan_selected"].astype(str).str.strip().str.lower()
df["plan_selected"] = df["plan_selected"].replace({
    "basic": "Basic",
    "pro": "Pro",
    "premium": "Premium",
    "prem": "Premium",
    "unknownplan": "Unknown",
    "nan": np.nan
}).str.title()

# Standardise gender values
df["gender"] = df["gender"].astype(str).str.strip().str.lower()
df["gender"] = df["gender"].replace({
    "male": "Male",
    "female": "Female",
    "non-binary": "Non-Binary",
    "other": "Other",
    "123": "Unknown",
    "nan": np.nan
}).replace({"male":"Male","female":"Female"}).str.title()

# Standardise marketing_opt_in
df["marketing_opt_in"] = df["marketing_opt_in"].astype(str).str.strip().str.title().replace({
    "Nil": "No",
    "Nan": np.nan
})

# 4) Age: convert to numeric and remove unrealistic values
df["age"] = pd.to_numeric(df["age"], errors="coerce")
df.loc[(df["age"] < 13) | (df["age"] > 100), "age"] = np.nan

# 5) Fill missing values
df["email"] = df["email"].fillna("Missing")
df["name"] = df["name"].fillna("Unknown")
df["region"] = df["region"].fillna("Unknown")
df["source"] = df["source"].fillna("Unknown")
df["plan_selected"] = df["plan_selected"].fillna("Unknown")
df["gender"] = df["gender"].fillna("Unknown")
df["marketing_opt_in"] = df["marketing_opt_in"].fillna("Unknown")
df["age"] = df["age"].fillna(df["age"].median())

df.head()


C:\Users\khali\AppData\Local\Temp\ipykernel_22032\1381275055.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["signup_date"] = pd.to_datetime(df["signup_date"], errors="coerce", dayfirst=True)


,customer_id,name,email,signup_date,source,region,plan_selected,marketing_opt_in,age,gender
0,CUST00000,Joshua Bryant,Missing,NaT,Instagram,Unknown,Basic,No,34.0,Female
1,CUST00001,Nicole Stewart,nicole1@example.com,2024-01-02,Linkedin,West,Basic,Yes,29.0,Male
2,CUST00002,Rachel Allen,rachel2@example.com,2024-01-03,Google,North,Premium,Yes,34.0,Non-Binary
3,CUST00003,Zachary Sanchez,zachary3@mailhub.org,2024-01-04,Youtube,Unknown,Pro,No,40.0,Male
4,CUST00004,Unknown,matthew4@mailhub.org,2024-01-05,Linkedin,West,Premium,No,25.0,Other


In [5]:
# Missing values after cleaning
df.isnull().sum()


customer_id         0
name                0
email               0
signup_date         6
source              0
region              0
plan_selected       0
marketing_opt_in    0
age                 0
gender              0
dtype: int64

In [6]:
# Missing values BEFORE cleaning (reload original)
df_raw = pd.read_csv("customer_signups.csv")
missing_counts = df_raw.isnull().sum()
missing_pct = (missing_counts / len(df_raw) * 100).round(1)

missing_counts, missing_pct


(customer_id          2
 name                 9
 email               34
 signup_date          2
 source               9
 region              30
 plan_selected        8
 marketing_opt_in    10
 age                 12
 gender               8
 dtype: int64,
 customer_id          0.7
 name                 3.0
 email               11.3
 signup_date          0.7
 source               3.0
 region              10.0
 plan_selected        2.7
 marketing_opt_in     3.3
 age                  4.0
 gender               2.7
 dtype: float64)

In [7]:
#Sign-ups per week

df["week"] = df["signup_date"].dt.to_period("W")
df.groupby("week").size()

week
2024-01-01/2024-01-07    6
2024-01-08/2024-01-14    7
2024-01-15/2024-01-21    7
2024-01-22/2024-01-28    7
2024-01-29/2024-02-04    8
2024-02-05/2024-02-11    7
2024-02-12/2024-02-18    7
2024-02-19/2024-02-25    7
2024-02-26/2024-03-03    7
2024-03-04/2024-03-10    7
2024-03-11/2024-03-17    7
2024-03-18/2024-03-24    6
2024-03-25/2024-03-31    6
2024-04-01/2024-04-07    7
2024-04-08/2024-04-14    7
2024-04-15/2024-04-21    7
2024-04-22/2024-04-28    7
2024-04-29/2024-05-05    6
2024-05-06/2024-05-12    7
2024-05-13/2024-05-19    7
2024-05-20/2024-05-26    7
2024-05-27/2024-06-02    7
2024-06-03/2024-06-09    6
2024-06-10/2024-06-16    6
2024-06-17/2024-06-23    7
2024-06-24/2024-06-30    7
2024-07-01/2024-07-07    7
2024-07-08/2024-07-14    7
2024-07-15/2024-07-21    6
2024-07-22/2024-07-28    7
2024-07-29/2024-08-04    7
2024-08-05/2024-08-11    6
2024-08-12/2024-08-18    7
2024-08-19/2024-08-25    7
2024-08-26/2024-09-01    7
2024-09-02/2024-09-08    7
2024-09-09/2024-09-15  

In [ ]:

df["plan_selected"].value_counts()


plan_selected
Premium    99
Pro        93
Basic      92
Unknown    14
Name: count, dtype: int64

In [12]:
df["region"].value_counts()



region
North      65
East       61
South      58
West       45
Central    39
Unknown    30
Name: count, dtype: int64

In [13]:
df["source"].value_counts()



source
Youtube      58
Google       50
Referral     49
Instagram    48
Facebook     40
Linkedin     38
Unknown       9
??            6
Name: count, dtype: int64

In [14]:
pd.crosstab(df["gender"], df["marketing_opt_in"])


marketing_opt_in,No,Unknown,Yes
gender,,,
Female,47,1,44
Male,51,3,37
Non-Binary,20,3,19
Other,32,3,24
Unknown,7,0,7


In [16]:
df["age"].describe()



count    298.000000
mean      35.466443
std       10.677104
min       21.000000
25%       25.000000
50%       34.000000
75%       40.000000
max       60.000000
Name: age, dtype: float64

In [17]:
df["age"].isnull().sum()


np.int64(0)